In [1]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from unet import UNet
from data_vis import plot_img_and_mask
from dataset import BasicDataset

In [2]:
def mask_to_image(mask):
    return Image.fromarray((mask * 255).astype(np.uint8))

def predict_img(net,
                full_img,
                device,
                scale_factor=0.3,
                out_threshold=0.5):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img, scale_factor))

    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)
    output = output.squeeze(0)
    tf = transforms.Compose(
            [
                transforms.ToPILImage()
                # transforms.Resize(full_img.size[1]),
                # transforms.ToTensor()
            ]
        )

    output = tf(output.cpu())
    return output
def mask_to_image(mask):
    return Image.fromarray((mask * 255))


In [3]:
for i in range(1,80):
    PREFIX="01"
    MODELDIR="/root/depthEstima/Demo/pytorch-nyuv2/checkpoints/CP_epoch{}.pth".format(i)
    IMGPATH="./0017.png"
    net = UNet(n_channels=3, n_classes=1)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device=device)
    net.load_state_dict(torch.load(MODELDIR, map_location=device))

    img = Image.open(IMGPATH)
    img.show()
    outPut = predict_img(net=net,
                            full_img=img,
                            scale_factor=0.3,
                            out_threshold=1,
                            device=device)
    outPut.save("./outTest/{}_{}.png".format(PREFIX,i))
